**NSP (Next Sentence Prediction) Logic**

In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.2 MB/s eta 0:00:00


In [ ]:
from transformers import BertTokenizer, BertForPreTraining
import torch

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForPreTraining.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
text = ("After Abraham Lincoln won the November 1860 presidential [MASK] on an "
        "anti-slavery platform, an initial seven slave states declared their "
        "secession from the country to form the Confederacy.")
text2 = ("War broke out in April 1861 when secessionist forces [MASK] Fort "
         "Sumter in South Carolina, just over a month after Lincoln's "
         "inauguration.")

In [ ]:
inputs = tokenizer(text, text2, return_tensors="pt")

In [ ]:
inputs
# 0 represents 1st sentence, 1 represents 2nd sentence

{'input_ids': tensor([[  101,  2044,  8181,  5367,  2180,  1996,  2281,  7313,  4883,   103,
          2006,  2019,  3424,  1011,  8864,  4132,  1010,  2019,  3988,  2698,
          6658,  2163,  4161,  2037, 22965,  2013,  1996,  2406,  2000,  2433,
          1996, 18179,  1012,   102,  2162,  3631,  2041,  1999,  2258,  6863,
          2043, 22965,  2923,  2749,   103,  3481,  7680,  3334,  1999,  2148,
          3792,  1010,  2074,  2058,  1037,  3204,  2044,  5367,  1005,  1055,
         17331,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
outputs = model(**inputs)
outputs.keys()
#seq rel logits is for NSP

odict_keys(['prediction_logits', 'seq_relationship_logits'])

In [ ]:
outputs.seq_relationship_logits

tensor([[ 6.0843, -5.6813]], grad_fn=<AddmmBackward0>)

In [ ]:
argmax = torch.argmax(outputs.seq_relationship_logits)

In [ ]:
argmax
#index 0 represents ISNEXT i.e. sentence B follows sentence A
#index 1 represents NotNext


tensor(0)

In [ ]:
argmax.item()

0

In [ ]:
'NotNext' if argmax.item() else 'IsNext'

'IsNext'

**NSP Traning Logic**

In [ ]:
from transformers import BertTokenizer, BertForNextSentencePrediction
import torch

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')

In [ ]:
text = ("After Abraham Lincoln won the November 1860 presidential [MASK] on an "
        "anti-slavery platform, an initial seven slave states declared their "
        "secession from the country to form the Confederacy.")
text2 = ("War broke out in April 1861 when secessionist forces [MASK] Fort "
         "Sumter in South Carolina, just over a month after Lincoln's "
         "inauguration.")

In [ ]:
inputs = tokenizer(text, text2, return_tensors='pt')
inputs

{'input_ids': tensor([[  101,  2044,  8181,  5367,  2180,  1996,  2281,  7313,  4883,   103,
          2006,  2019,  3424,  1011,  8864,  4132,  1010,  2019,  3988,  2698,
          6658,  2163,  4161,  2037, 22965,  2013,  1996,  2406,  2000,  2433,
          1996, 18179,  1012,   102,  2162,  3631,  2041,  1999,  2258,  6863,
          2043, 22965,  2923,  2749,   103,  3481,  7680,  3334,  1999,  2148,
          3792,  1010,  2074,  2058,  1037,  3204,  2044,  5367,  1005,  1055,
         17331,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
labels = torch.LongTensor([0])
labels

tensor([0])

In [ ]:
outputs = model(**inputs, labels=labels)
outputs.keys()

odict_keys(['loss', 'logits'])

In [ ]:
outputs.loss

tensor(7.7486e-06, grad_fn=<NllLossBackward0>)

In [ ]:
x=0.0002-7.7486e-06
print(x)

0.0001922514


**NSP Training**

In [ ]:
paragraphs=[]

paragraphs.append("Mount Everest, also known as Sagarmatha in Nepal and Chomolungma in Tibet, is the highest peak on Earth. It stands majestically at an elevation of 8,848.86 meters (29,031.7 feet) above sea level. Climbers from around the world attempt to conquer its treacherous slopes, facing extreme cold, thin air, and unpredictable weather conditions.")


paragraphs.append("Bananas, on the other hand, are a common fruit found in grocery stores and fruit bowls. An average banana measures approximately 7.5 inches in length. They are yellow, slightly curved, and have a sweet taste when ripe. Whether you're enjoying a banana as a snack or using it in a smoothie, it's a versatile and nutritious fruit.")

paragraphs.append("Now, let's playfully compare Mount Everest's height to bananas. If we stack bananas vertically, each measuring 7.5 inches, it would take approximately 46,449 bananas to reach the summit of Everest. Of course, this is just a fun way to visualize its immense height!")




In [ ]:
paragraphs[0].split('.')
#we want to get rid of empty ''

['Mount Everest, also known as Sagarmatha in Nepal and Chomolungma in Tibet, is the highest peak on Earth',
 ' It stands majestically at an elevation of 8,848',
 '86 meters (29,031',
 '7 feet) above sea level',
 ' Climbers from around the world attempt to conquer its treacherous slopes, facing extreme cold, thin air, and unpredictable weather conditions',
 '']

In [ ]:
bag = [item for sentence in paragraphs for item in sentence.split('.') if item != '']
bag_size = len(bag)
print(bag_size)

14


In [ ]:
import random

sentence_a = []
sentence_b = []
label = []
for sentence in paragraphs:
    texts = [item for item in sentence.split('.') if item != '']
    num_sentences = len(texts)
    if num_sentences > 1:
        start = random.randint(0, num_sentences-2)
        #num_sentences-2: If we have 2 sentences then it would be (0,0). By chance , (0,1) then if start =1 then start +1 will be out of bound.
        # label=0: isnext, label=1: notnext
        if random.random() >= 0.5:
            # this is IsNextSentence
            sentence_a.append(texts[start])
            sentence_b.append(texts[start+1])
            label.append(0)
        else:
            index = random.randint(0, bag_size-1)
            # this is NotNextSentence
            sentence_a.append(texts[start])
            sentence_b.append(bag[index])
            label.append(1)




In [ ]:
for i in range(3):
    print(label[i])
    print(sentence_a[i] + '\n---')
    print(sentence_b[i] + '\n')

1
7 feet) above sea level
---
 Climbers from around the world attempt to conquer its treacherous slopes, facing extreme cold, thin air, and unpredictable weather conditions

0
 An average banana measures approximately 7
---
5 inches in length

0
5 inches, it would take approximately 46,449 bananas to reach the summit of Everest
---
 Of course, this is just a fun way to visualize its immense height!



**To avoid splitting decimal numbers**

In [ ]:
paragraphs[1].split('.')

['Bananas, on the other hand, are a common fruit found in grocery stores and fruit bowls',
 ' An average banana measures approximately 7',
 '5 inches in length',
 ' They are yellow, slightly curved, and have a sweet taste when ripe',
 " Whether you're enjoying a banana as a snack or using it in a smoothie, it's a versatile and nutritious fruit",
 '']

In [ ]:
import re
#split the string based on dot if not preceded or followed by digits i.e. decimal number
sentences = re.split(r'(?<!\d)\.(?!\d)', paragraphs[1])
sentences=[item for item in sentences if item!='']
sentences

['Bananas, on the other hand, are a common fruit found in grocery stores and fruit bowls',
 ' An average banana measures approximately 7.5 inches in length',
 ' They are yellow, slightly curved, and have a sweet taste when ripe',
 " Whether you're enjoying a banana as a snack or using it in a smoothie, it's a versatile and nutritious fruit"]

In [ ]:
paragraphs=[]

paragraphs.append("Mount Everest, also known as Sagarmatha in Nepal and Chomolungma in Tibet, is the highest peak on Earth. It stands majestically at an elevation of 8,848.86 meters (29,031.7 feet) above sea level. Climbers from around the world attempt to conquer its treacherous slopes, facing extreme cold, thin air, and unpredictable weather conditions.")


paragraphs.append("Bananas, on the other hand, are a common fruit found in grocery stores and fruit bowls. An average banana measures approximately 7.5 inches in length. They are yellow, slightly curved, and have a sweet taste when ripe. Whether you're enjoying a banana as a snack or using it in a smoothie, it's a versatile and nutritious fruit.")

paragraphs.append("Now, let's playfully compare Mount Everest's height to bananas. If we stack bananas vertically, each measuring 7.5 inches, it would take approximately 46,449 bananas to reach the summit of Everest. Of course, this is just a fun way to visualize its immense height!")



In [ ]:
bag = [item for sentence in paragraphs for item in (re.split(r'(?<!\d)\.(?!\d)', sentence)) if item != '']
bag_size = len(bag)
print(bag_size)

10


In [ ]:
bag

['Mount Everest, also known as Sagarmatha in Nepal and Chomolungma in Tibet, is the highest peak on Earth',
 ' It stands majestically at an elevation of 8,848.86 meters (29,031.7 feet) above sea level',
 ' Climbers from around the world attempt to conquer its treacherous slopes, facing extreme cold, thin air, and unpredictable weather conditions',
 'Bananas, on the other hand, are a common fruit found in grocery stores and fruit bowls',
 ' An average banana measures approximately 7.5 inches in length',
 ' They are yellow, slightly curved, and have a sweet taste when ripe',
 " Whether you're enjoying a banana as a snack or using it in a smoothie, it's a versatile and nutritious fruit",
 "Now, let's playfully compare Mount Everest's height to bananas",
 ' If we stack bananas vertically, each measuring 7.5 inches, it would take approximately 46,449 bananas to reach the summit of Everest',
 ' Of course, this is just a fun way to visualize its immense height!']

In [ ]:
import re
import random

sentence_a = []
sentence_b = []
label = []
for sentence in paragraphs:
    texts = re.split(r'(?<!\d)\.(?!\d)', sentence)
    texts=[item for item in texts if item!='']
    print(texts)
    num_sentences = len(texts)
    if num_sentences > 1:
        start = random.randint(0, num_sentences-2)
        #num_sentences-2: If we have 2 sentences then it would be (0,0). By chance , (0,1) then if start =1 then start +1 will be out of bound.
        # label=0: isnext, label=1: notnext
        if random.random() >= 0.5:
            # this is IsNextSentence
            sentence_a.append(texts[start])
            sentence_b.append(texts[start+1])
            label.append(0)
        else:
            index = random.randint(0, bag_size-1)
            # this is NotNextSentence
            sentence_a.append(texts[start])
            sentence_b.append(bag[index])
            label.append(1)





['Mount Everest, also known as Sagarmatha in Nepal and Chomolungma in Tibet, is the highest peak on Earth', ' It stands majestically at an elevation of 8,848.86 meters (29,031.7 feet) above sea level', ' Climbers from around the world attempt to conquer its treacherous slopes, facing extreme cold, thin air, and unpredictable weather conditions']
['Bananas, on the other hand, are a common fruit found in grocery stores and fruit bowls', ' An average banana measures approximately 7.5 inches in length', ' They are yellow, slightly curved, and have a sweet taste when ripe', " Whether you're enjoying a banana as a snack or using it in a smoothie, it's a versatile and nutritious fruit"]
["Now, let's playfully compare Mount Everest's height to bananas", ' If we stack bananas vertically, each measuring 7.5 inches, it would take approximately 46,449 bananas to reach the summit of Everest', ' Of course, this is just a fun way to visualize its immense height!']


In [ ]:
label

[0, 0, 1]

In [ ]:
for i in range(3):
    print(label[i])
    print(sentence_a[i] + '\n---')
    print(sentence_b[i] + '\n')

0
 It stands majestically at an elevation of 8,848.86 meters (29,031.7 feet) above sea level
---
 Climbers from around the world attempt to conquer its treacherous slopes, facing extreme cold, thin air, and unpredictable weather conditions

0
Bananas, on the other hand, are a common fruit found in grocery stores and fruit bowls
---
 An average banana measures approximately 7.5 inches in length

1
 If we stack bananas vertically, each measuring 7.5 inches, it would take approximately 46,449 bananas to reach the summit of Everest
---
Now, let's playfully compare Mount Everest's height to bananas



In [ ]:
from transformers import BertTokenizer, BertForPreTraining,BertForNextSentencePrediction
import torch


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')

In [ ]:
inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

In [ ]:
inputs.token_type_ids

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])

In [ ]:
inputs.token_type_ids.shape

torch.Size([3, 512])

In [ ]:
inputs.token_type_ids[0]
#initial 0=sentence a
#initial 1=sentence b
#0 followed by 1=padded tokens

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
torch.LongTensor(label)

tensor([0, 0, 1])

In [ ]:
torch.LongTensor([label])

tensor([[0, 0, 1]])

In [ ]:
inputs['labels'] = torch.LongTensor([label]).T

In [ ]:
inputs.labels

tensor([[0],
        [0],
        [1]])

In [ ]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [ ]:
len(inputs.input_ids)

3

In [ ]:
inputs.attention_mask[0]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

**Fine-Tuning**

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
dataset = Dataset(inputs)

In [ ]:
loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

BertForNextSentencePrediction(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=5e-6)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from tqdm import tqdm

epochs = 20
for epoch in range(epochs):
    loop = tqdm(loader, leave=True)
    for batch in loop:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        labels=labels)
        loss = outputs.loss
        loss.backward()
        optim.step()
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/3 [00:00<?, ?it/s]<ipython-input-129-d87c8321e905>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Epoch 19: 100%|██████████| 3/3 [00:00<00:00, 23.74it/s, loss=0.000277]


In [ ]:
outputs = model(input_ids, attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        labels=labels)
outputs.keys()

odict_keys(['loss', 'logits'])